# Experiments.ipynb

This contains poorly maintained code for

 - Interleaved Model Resynthesis
 - LossPred
 - Boundary Evaluation

In [ ]:
import torch
import torch.nn.functional as F
import torchaudio
import sys
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import sys
import time
import matplotlib.ticker as ticker
import os
import csv
os.environ["CUDA_VISIBLE_DEVICES"]="7"
sys.path.append('/home/abaade/MAE-AST-Playground')
import fairseq
from fairseq import models, checkpoint_utils
import json

In [ ]:
ckpt_path = "/data/scratch/abaade/cluster/hubert_large_ll60k.pt"
hubert_large_models, _, _ = fairseq.checkpoint_utils.load_model_ensemble_and_task([ckpt_path])
hubert_large_model = hubert_large_models[0]
hubert_large_model = hubert_large_model.eval().cuda()

ckpt_path = "/data/scratchre/abaade/hubert/hubert_base_ls960.pt"
hubert_base_models, _, _ = fairseq.checkpoint_utils.load_model_ensemble_and_task([ckpt_path])
hubert_base_model = hubert_base_models[0]
hubert_base_model = hubert_base_model.eval().cuda()

In [ ]:
state_dict = torch.load('/data/scratch/abaade/cluster/SegmentalSSE/exp/alan_a40_ls_runAve0_crf1_crfWei1_bceWei0_freezeSeg0_pretrain_faTrain0_posWeight3_useFa0_segQuan0.95_ancimage_cls1_comLayer1_9_clsPolicy0.00001_featPolicy-10202_clsMatch4_featMatch0_aweFreeze0/bundle.pth')
ckpt_path = "/data/scratch/abaade/hubert/hubert_base_ls960.pt"
mean_base_models, _, _ = fairseq.checkpoint_utils.load_model_ensemble_and_task([ckpt_path])
mean_base_model_new2 = mean_base_models[0]
mean_base_model_new2 = mean_base_model_new2.eval().cuda()
mean_base_model_new2.half()

In [ ]:
from examples.data2vec.models import data2vec2

In [ ]:

d2v2_model_dev, _, _ = checkpoint_utils.load_model_ensemble_and_task(
    ["/data/scratch/abaade/data2vec2/base_libri.pt"],
)
d2v2_model_dev = d2v2_model_dev[0].cuda().eval().half()
state_dict = torch.load('/data/scratch/abaade/cluster/SegmentalSSE/exp/alan_a40_ls_runAve0_crf1_crfWei1_bceWei0_freezeSeg0_pretrain_faTrain0_posWeight3_useFa0_segQuan0.95_ancimage_cls1_comLayer1_9_clsPolicy0.00001_featPolicy-2120_clsMatch4_featMatch0_aweFreeze0/bundle.pth')
d2v2_model_dev.load_state_dict({k[len('model.'):]: v for k,v in state_dict['model_seg'].items()})

In [ ]:
from models.classifier import Classifier, get_features
import omegaconf
cfg = omegaconf.OmegaConf.load('./config/conf.yaml')
from utils.misc import load_from_checkpoint

In [ ]:
def add_mean(x, lst):
    # if len(x) == 0:
    #     return
    # elif len(x) == 1:
    #     lst.append(0.0)
    # else:
    #     for _ in range(len(x)):
    #         lst.append(torch.stack(x).var(dim=0).mean().detach().cpu().item())
    
    if len(x) == 5:
        for _ in range(len(x)):
            lst.append(torch.stack(x).var(dim=0).mean().detach().cpu().item())

def phone_class_file(audio_file, pred_override=None, p=0.5):
    with torch.no_grad():
        wavs, sr = torchaudio.load(audio_file)
        assert sr == 16000
        wavs = wavs.cuda()

        results = model.forward(wavs, padding_mask=None, features_only=True, mask=False)
        
        if pred_override is None:
            results["layer_results"] = results["hidden_states"]
            features = get_features(results, cfg.mode)
            preds = classifier(features).squeeze()
            preds = F.sigmoid(preds)
            preds = preds > p
        else:
            preds = torch.tensor(pred_override)
        
        cur_a = []
        cur_b = []
        outs_a = []
        outs_b = []
        layer_result = results['hidden_states'][8][0].transpose(0,1)[0]
        
        lr = layer_result
        z = (lr - lr.mean(dim=-1, keepdim=True))/lr.std(dim=-1, keepdim=True)
        layer_result = z
        
        print(preds.shape, layer_result.shape)
        for idx in range(layer_result.size(0)):
            cur_x = layer_result[idx]
            
            cur_a.append(cur_x)
            if preds[idx]:
                add_mean(cur_a, outs_a)
                add_mean(cur_b, outs_b)
                cur_a = []
                cur_b = []
            cur_b.append(cur_x)
                    
        add_mean(cur_a, outs_a)
        add_mean(cur_b, outs_b)

    
    return preds, outs_a, outs_b

In [ ]:
def get_word_ali(raw_ali, fps=50):
    """
    raw_ali is a string like 'start1__word1__end1 start2__word2__end2 ...'
    """
    data = []
    meta_toks = raw_ali.split()
    for meta_tok in meta_toks:
        toks = meta_tok.split('__')
        if len(toks) == 3:
            data.append((float(toks[0])*fps, float(toks[2])*fps, toks[1]))
    return data

In [ ]:
def plot_featmap(feat, word_ali, start_in_sec=0, fps=50, image_already=None,
                 boundary=None, cmap=None, figsize=(15,15), save_to=None,
                 extend_words=False, do_yaxis=True, kill_last=False,
                ):    
    fig, ax = plt.subplots(figsize=figsize)
    if image_already is not None:
        ax.imshow(image_already, cmap=cmap) # TODO why inversing it?
    else:
        ax.imshow(feat@feat.transpose(1,0), cmap=cmap) # TODO why inversing it?
    if word_ali != None:
        ticks = []
        minor_ticks = []
        labels = []
        for i, item in enumerate(word_ali):
            if item[0] - fps*start_in_sec >= -1:
                ticks.append(item[0] - fps*start_in_sec)
                labels.append(item[2].lower())
                minor_ticks.append((item[1] + item[0])//2-fps*start_in_sec)
        ticks.append(word_ali[-1][1])

        ax.xaxis.set_major_locator(ticker.FixedLocator(ticks))
        ax.xaxis.set_minor_locator(ticker.FixedLocator(minor_ticks))

        ax.xaxis.set_major_formatter(ticker.NullFormatter())
        ax.xaxis.set_minor_formatter(ticker.FixedFormatter(labels))

        for i, tick in enumerate(ax.xaxis.get_minor_ticks()):
            if i % 2 == 0:
                tick.set_pad(17)  # Increase or decrease the value to adjust the height
            else:
                tick.set_pad(3)  # Increase or decrease the value to adjust the height
            tick.tick1line.set_markersize(0)
            tick.tick2line.set_markersize(0)
            tick.label1.set_horizontalalignment('center')

        if do_yaxis:
            ax.yaxis.set_major_locator(ticker.FixedLocator(ticks))
            ax.yaxis.set_minor_locator(ticker.FixedLocator(minor_ticks))

            ax.yaxis.set_major_formatter(ticker.NullFormatter())
            ax.yaxis.set_minor_formatter(ticker.FixedFormatter(labels))
        else:
            ax.yaxis.set_major_locator(ticker.NullLocator())
            ax.yaxis.set_minor_locator(ticker.NullLocator())

            ax.yaxis.set_major_formatter(ticker.NullFormatter())
            ax.yaxis.set_minor_formatter(ticker.NullFormatter())


        for tick in ax.yaxis.get_minor_ticks():
            tick.tick1line.set_markersize(0)
            tick.tick2line.set_markersize(0)
            tick.label1.set_horizontalalignment('center')
        ax.tick_params(axis='y', which='minor', rotation=90)
        if extend_words:
            (ymin, ymax) = ax.get_ylim()
            for b in (ticks[:-1] if kill_last else ticks):
                ax.vlines(x=b, ymin = ymin, ymax=((ymin+ymax)*0.93 if extend_words else ymax), colors='red', ls='-', lw=3)
        if boundary is not None:
            (ymin, ymax) = ax.get_ylim()
            for b in boundary:
                ax.vlines(x=b, ymin = ((ymin+ymax)*0.93 if extend_words else ymin), ymax=ymax, colors='#AA0000', ls='-', lw=3)
                
                 
        for label in ax.get_xticklabels(minor=True):
            label.set_fontsize(20)
        ax.tick_params(axis='x', which='major', length=20, width=3, colors='#0000AA')

                
        if save_to is not None:
            plt.tight_layout()
#             plt.gca().set_axis_off()
            plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
                        hspace = 0, wspace = 0)
            plt.margins(0,0)
            plt.savefig(save_to, bbox_inches='tight')
            plt.clf()
        else:
            plt.show()
            
from PIL import Image
def concatenate_vertically(image1_path, image2_path, output_path):
    image1 = Image.open(image1_path)
    image2 = Image.open(image2_path)

    if image1.width != image2.width:
        raise ValueError("Images must have the same width for vertical concatenation.")

    new_image = Image.new("RGB", (image1.width, image1.height + image2.height))
    new_image.paste(image1, (0, 0))
    new_image.paste(image2, (0, image1.height))
    new_image.save(output_path)

## Annoying Imports

In [8]:
sys.path.append('/home/abaade/MAE-AST-Playground/examples/hubert/simple_kmeans/')

from examples.hubert.simple_kmeans.alan_good_kmeans import HubertFeatureReader, ApplyKmeans

In [9]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
# import seaborn as sns
import torchaudio
import time
import soundfile as sf
import sys
sys.path.append('/home/abaade/MAE-AST-Playground/')
sys.path.append('/home/abaade/MAE-AST-Playground/examples')
import examples
sys.path.append('/home/abaade/MAE-AST-Playground/examples/data2vec')
sys.path.append('/home/abaade/MAE-AST-Playground/examples/data2vec/models')

from fairseq import checkpoint_utils

import data2vec2

In [ ]:
hubert_akm = ApplyKmeans('/data/scratch/abaade/cluster/hubert_base_ls960_L9_km500.bin')

## Twist

In [ ]:
import sys
import re
sys.path.append('/home/abaade/textlesslib')
sys.path.append('/home/abaade/textlesslib/examples/twist')
sys.path.append('/home/abaade/textlesslib/textless/data')
sys.path.append('/home/abaade/textlesslib/textless')

from textless.data.speech_encoder import SpeechEncoder

In [ ]:
dense_model, quantizer_model, vocab = "mhubert-base-25hz", "kmeans", 500
encoder_25hz = SpeechEncoder.by_name(
    dense_model_name = dense_model,
    quantizer_model_name = quantizer_model,
    vocab_size = vocab,
    deduplicate=False,
    need_f0=False,
    add_bos_eos=False,
).eval().cuda()

In [ ]:
from transformers import AutoModelForCausalLM
twist_lm_model = AutoModelForCausalLM.from_pretrained('/home/abaade/textlesslib/textlesslibexamples/twist/ckpts/TWIST-350M/')
# twist_lm_model = AutoModelForCausalLM.from_pretrained('/data/scratch/abaade/textlesslib/TWIST-1.3B/')
twist_lm_model = twist_lm_model.cuda()
twist_lm_model.eval()

In [ ]:

ours_ckpt_mindist = '/data/scratch/abaade/cluster/models/syl_d2v2_LibriSpeech_bulk_2212_24576_agg_8192_equal_compute_april_22/checkpoints/checkpoint_last.pt'

ours_lm_model_mindist, _, _ = fairseq.checkpoint_utils.load_model_ensemble_and_task([ours_ckpt_mindist])
ours_lm_model_mindist = ours_lm_model_mindist[0]
ours_lm_model_mindist = ours_lm_model_mindist.cuda()
ours_lm_model_mindist = ours_lm_model_mindist.eval()
ours_lm_model_mindist = ours_lm_model_mindist.half()

2025-02-23 13:38:15 | INFO | fairseq.tasks.cluster_alan_pretraining | current directory is /home/abaade/phone-seg
2025-02-23 13:38:15 | INFO | fairseq.tasks.cluster_alan_pretraining | ClusterAlanPretrainingConfig Config {'_name': 'cluster_alan_pretraining', 'data': '/data/scratch/abaade/cluster/tsv/libri_light_bulk_mhubert25hz/', 'sample_rate': 16000, 'normalize': True, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 208005, 'min_sample_size': 0, 'min_keep_phone_size': 50, 'random_crop': True, 'feature_type': 'wav', 'encodec_rate': 75, 'full_prompt_type': 'norm', 'pred_prompt_type': 'encodec', 'reducer_path': None, 'tsv_replace_source': None, 'tsv_replace_target': None, 'bpe_path': '/data/scratch/abaade/cluster/acoustic_bpe/tokenizer_4096.json', 'vocab_size': 4096}


In [ ]:
ours_ckpt_resynthesis = '/data/scratch/abaade/cluster/models/syl_d2v2_interleaved5_LibriSpeech_bulk_2116_24576_8192/checkpoints/checkpoint_last.pt'

ours_lm_model_resynthesis, _, _ = fairseq.checkpoint_utils.load_model_ensemble_and_task([ours_ckpt_resynthesis])
ours_lm_model_resynthesis = ours_lm_model_resynthesis[0]
ours_lm_model_resynthesis = ours_lm_model_resynthesis.cuda()
ours_lm_model_resynthesis = ours_lm_model_resynthesis.eval()
ours_lm_model_resynthesis = ours_lm_model_resynthesis.half()

2025-02-23 13:27:28 | INFO | fairseq.tasks.cluster_alan_interleaved_pretraining | current directory is /home/abaade/phone-seg
2025-02-23 13:27:28 | INFO | fairseq.tasks.cluster_alan_interleaved_pretraining | ClusterAlanPretrainingConfig Config {'_name': 'cluster_alan_interleaved_pretraining', 'data': '/data/scratch/abaade/cluster/tsv/libri_light_full_interleave/', 'sample_rate': 16000, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 208005, 'min_sample_size': 0, 'min_keep_phone_size': 50, 'random_crop': True, 'feature_type': 'wav', 'encodec_rate': 75, 'full_prompt_type': 'norm', 'pred_prompt_type': 'encodec', 'vocab_size_short': 1024, 'vocab_size_long': 8192, 'interleave_strat': 5, 'long_dropout': 0.0, 'reducer_path': '/saltpool0/scratch/abaade/cluster/LibriSpeech_bulk_2116_24576_agg_8192.npy', 'tsv_replace_source': 'LibriSpeech_bulk_2120_32768_w23_0019', 'tsv_replace_target': 'LibriSpeech_bulk_2116_24576', 'interleave_long_grouping': 1}


In [24]:
import importlib
importlib.reload(fairseq.data.audio.cluster_alan_dataset_interleaved)
from fairseq.data.audio.cluster_alan_dataset_interleaved import ClusterAlanDatasetInterleaved

## RESYNTHESIS

In [38]:
import IPython
import textless
dense_model, quantizer_model, vocab = "mhubert-base-25hz", "kmeans", 500
vocoder = textless.CodeHiFiGANVocoder.by_name(
    dense_model_name = dense_model,
    quantizer_model_name = quantizer_model,
    vocab_size = vocab
).eval().cuda()

Removing weight norm...
CodeHiFiGAN model loaded!


In [39]:
@torch.no_grad()
def resynth(tokens_mhubert, out_path='/home/abaade/temp.wav', max_len=None):
    out_audio = vocoder(tokens_mhubert, dur_prediction = True)
    out_audio = out_audio[:max_len] if max_len is not None else out_audio
    torchaudio.save(out_path, out_audio.cpu().unsqueeze(0)[:,:], 16000)

In [29]:
from fairseq.data.audio.cluster_alan_dataset_interleaved import ClusterAlanDatasetInterleaved
# interleaved_dataset = ClusterAlanDatasetInterleaved(None, interleave_strat=3, vocab_size_long=4096)
interleaved_dataset = ClusterAlanDatasetInterleaved(
    None, interleave_strat=5, 
#     vocab_size_long=2048, reducer_path='/saltpool0/scratch/abaade/cluster/LibriSpeech_bulk_2212_24576_agg_2048.npy',
    vocab_size_long=8192, reducer_path='/saltpool0/scratch/abaade/cluster/LibriSpeech_bulk_2116_24576_agg_8192.npy',
#     vocab_size_long=16384, reducer_path='/saltpool0/scratch/abaade/cluster/LibriSpeech_bulk_2120_24576_16384_w23.npy',
#     vocab_size_long=4096, reducer_path='/saltpool0/scratch/abaade/cluster/LibriSpeech_bulk_2120_24576_4096_w23.npy',
#     vocab_size_long=4096, reducer_path='/saltpool0/scratch/abaade/cluster/LibriSpeech_bulk_1120_16384_4096.npy',
    tsv_replace_source='QWRQWRQWRQWR',
    tsv_replace_target='QWRQWRQWRQWR',
)
task = "tSC"
mode = 'dev'
zs_dir = f'/data/scratch/abaade/cluster/{task}/'
zs_lengths_file = f'/data/scratch/abaade/cluster/{task}/{mode}.tsv'


audio_file = "/data/scratch/abaade/cluster/tSC/" + [
"0_138d5bfb-05cc-41e3-bf2c-fa85ebad14e2_correct.wav",
"1000_7ab5d7b2-f587-433f-b9b0-79a841cf0753_correct.wav",
"1001_7ab5d7b2-f587-433f-b9b0-79a841cf0753_incorrect.wav",
"1002_5a4e4ff0-d972-4991-8f91-bcf4dae1288e_correct.wav",
"1003_5a4e4ff0-d972-4991-8f91-bcf4dae1288e_incorrect.wav",
"1004_2d1c5e85-93b9-41a6-8c0f-f8e6035fe78e_correct.wav",
"1005_2d1c5e85-93b9-41a6-8c0f-f8e6035fe78e_incorrect.wav",
"100_57d5152a-77db-4156-8330-166edb36b0a2_correct.wav",
"1006_069fe406-47af-4c57-ae9c-2ffea2a0f7ee_correct.wav",
"1007_069fe406-47af-4c57-ae9c-2ffea2a0f7ee_incorrect.wav",
][5]

In [31]:
def nucleus_sample(logits, p=0.9, m=25):
    raw_probs = logits.softmax(dim=-1)
    sorted_indices = raw_probs.argsort(dim=-1, descending=True)
    too_small_indices = sorted_indices[1:][(raw_probs[sorted_indices].cumsum(-1) > p)[:-1]]
    nucleus_logits = logits.clone()
    nucleus_logits[too_small_indices] = -1000
    nucleus_logits[sorted_indices[m:]] = -1000
    nucleus_probs = nucleus_logits.softmax(dim=-1)
    return np.random.choice(np.arange(len(nucleus_probs)), p=nucleus_probs.cpu().numpy())

In [25]:
def temperature_sample(logits, t=1.0):
    scaled_logits = logits/t
    probs = scaled_logits.softmax(dim=-1)
    return np.random.choice(np.arange(len(probs)), p=probs.cpu().numpy())

In [ ]:
# ### RESYNTH WITH LEARNED STOP TOKENS


@torch.no_grad()
def forward_cache(model, full_tokens, pasts):
#     print(pasts.shape if pasts is not None else None)
    offset = 0 if pasts is None else pasts[0].size(-2)
    input_tokens = full_tokens[:, offset:]
    features = model.embed_weight[input_tokens]
    features = model.pos_embed(features, offset=offset)

    x, hidden_states, presents = model.encoder.forward_cache(features, padding_mask=None, pasts=pasts)
    logits = model.embed_weight @ x.transpose(-1, -2)
    pasts = presents if pasts is None else torch.cat([pasts, presents], dim=-2)
    
    return {'logits': logits}, pasts


@torch.no_grad()
def resynth_slow(sd_path, out_path):
#     raw_root = audio_file[:audio_file.rfind('/')]
#     raw_filename = audio_file[audio_file.rfind('/'):-len('.wav')] + '.npy'
#     mh_path = raw_root + '_clusters/mhubert_25hz' + raw_filename
#     sd_path = raw_root + '_clusters/LibriSpeech_bulk_2212_24576' + raw_filename
#     sd_path = raw_root + '_clusters/LibriSpeech_bulk_2120_24576_w23_0019' + raw_filename
#     sd_path = raw_root + '_clusters/sdhubert' + raw_filename
#     sd_path = '/data/scratch/abaade/cluster/librispeech_clusters/test/LibriSpeech_bulk_2212_24576/1995/1836/1995-1836-0006.npy'

    input_tokens_long = interleaved_dataset.reducer[np.load(sd_path)[0]] + interleaved_dataset.vocab_size_short + 1
#     input_tokens_long = np.array(output_long_tokens[1:]) + interleaved_dataset.vocab_size_short + 1
    
    magic_max_tokens = int(15*22.0)
    magic_true_num_short = 500
    magic_stop_idx = 1024
    magic_long_end_idx = interleaved_dataset.pad_idx-1
    
    long_token_idx = 0
    interleaved_tokens_list = []
    state = 'add_long'
#     'finish_up'
#     'add_short'
    finish_up_countdown = 1 if '2212' in sd_path else 2
    running_stop_prob = 1.

    pasts = None
    output_short_tokens = []
    
    for _ in range(magic_max_tokens+10):
        if state == 'add_long':
            interleaved_tokens_list.extend([input_tokens_long[long_token_idx], magic_long_end_idx])
            interleaved_tokens = torch.tensor(interleaved_tokens_list, device='cuda', dtype=int).view(1,-1)
            
            out_dict = ours_lm_model_resynthesis(interleaved_tokens, features_only=True)
#             out_dict, pasts = forward_cache(ours_lm_model_resynthesis, interleaved_tokens, pasts)
            
            logits = out_dict['logits']
            logits = logits.float()
            logits[:,magic_true_num_short:,:] = -100
            cur_short_logits = logits[0,:,-1].softmax(dim=0)
            next_prob_topk = cur_short_logits.topk(1)  # no stop token
            next_short_token = np.random.choice(
                next_prob_topk.indices.cpu().numpy(), 
                p=next_prob_topk.values.cpu().numpy()/next_prob_topk.values.cpu().numpy().sum()
            )
            
            output_short_tokens.append(next_short_token)
            interleaved_tokens_list.append(next_short_token)
            
            long_token_idx += 1
#             if long_token_idx == len(input_tokens_long):
#                 state = 'finish_up'
#             else:
            state = 'add_short'
        elif state == 'add_short':
            interleaved_tokens = torch.tensor(interleaved_tokens_list, device='cuda', dtype=int).view(1,-1)
            
            out_dict = ours_lm_model_resynthesis(interleaved_tokens, features_only=True)
#             out_dict, pasts = forward_cache(ours_lm_model_resynthesis, interleaved_tokens, pasts)
            logits = out_dict['logits']
            logits = logits.float()
            
            logits[:,magic_true_num_short:magic_stop_idx,:] = -100
            logits[:,magic_stop_idx+1:,:] = -100
            
            cur_short_logits = logits[0,:,-1].softmax(dim=0)
            running_stop_prob *= (1-cur_short_logits[magic_stop_idx])
            if running_stop_prob < (0.9 if long_token_idx == len(input_tokens_long) else 0.5):
                is_stop = True
            elif (len(output_short_tokens) / magic_max_tokens > long_token_idx / len(input_tokens_long)) and len(output_short_tokens) > 3:
                is_stop = True
                print('FORCED_END')
            else:
                is_stop = False

            logits[:,magic_stop_idx,:] = -100
            cur_short_logits = logits[0,:,-1].softmax(dim=0)
            next_prob_topk = cur_short_logits.topk(1)
            next_short_token = np.random.choice(
                next_prob_topk.indices.cpu().numpy(), 
                p=next_prob_topk.values.cpu().numpy()/next_prob_topk.values.cpu().numpy().sum()
            )
            
            if is_stop:
                running_stop_prob = 1.
                if long_token_idx == len(input_tokens_long):
                    output_short_tokens.append(next_short_token)
                    interleaved_tokens_list.append(next_short_token)
                    state = 'finish_up'
                else:
                    state = 'add_long'
            else:
                output_short_tokens.append(next_short_token)
                interleaved_tokens_list.append(next_short_token)
        elif state == 'finish_up':
            interleaved_tokens = torch.tensor(interleaved_tokens_list, device='cuda', dtype=int).view(1,-1)
            
            out_dict = ours_lm_model_resynthesis(interleaved_tokens, features_only=True)
#             out_dict, pasts = forward_cache(ours_lm_model_resynthesis, interleaved_tokens, pasts)
            logits = out_dict['logits']
            logits = logits.float()
            
            logits[:,magic_true_num_short:,:] = -100  # no stop token
            cur_short_logits = logits[0,:,-1].softmax(dim=0)
            next_prob_topk = cur_short_logits.topk(1)
            next_short_token = np.random.choice(
                next_prob_topk.indices.cpu().numpy(), 
                p=next_prob_topk.values.cpu().numpy()/next_prob_topk.values.cpu().numpy().sum()
            )
            
            output_short_tokens.append(next_short_token)
            interleaved_tokens_list.append(next_short_token)
            
            finish_up_countdown -= 1
            if finish_up_countdown == 0:
                break
        
    resynth(torch.tensor(output_short_tokens).cuda(), out_path)

In [ ]:
in_tsv = '/data/scratch/abaade/valle/evaluation/wer/data/test_4_to_10.tsv'
out_dir = '/data/scratch/abaade/valle/evaluation/output_audio/cluster_test_1120_4096a/'
ltr_tsv = '/data/scratch/abaade/valle/evaluation/wer/data/test_4_to_10.ltr'
in_npy = '/data/scratch/abaade/cluster/librispeech_clusters/test/LibriSpeech_bulk_2116_24576/'
# in_npy = '/data/scratch/abaade/cluster/librispeech_clusters/test/LibriSpeech_bulk_2120_24576_w23_0019/'
# in_npy = '/data/scratch/abaade/cluster/librispeech_clusters/test/sdhubert/'
# in_npy = '/data/scratch/abaade/cluster/librispeech_clusters/test/LibriSpeech_bulk_1120_16384_sdh_parity/'

with open(in_tsv, 'r') as f:
    next(f)
    for line in tqdm(f):
        in_path = line.split()[0]
        in_name = in_path[len('/data/scratch/pyp/datasets/LibriSpeech/data/test-clean/'):-len('.flac')]
        in_path_npy = in_npy + in_name + '.npy'
        out_path = out_dir + in_name + '.flac'
        print(in_path_npy)
        print(out_path)
        os.makedirs(out_path[:out_path.rfind('/')], exist_ok=True)
        resynth_slow(in_path_npy, out_path)
#         resynth_sdhubert_slow(in_path_npy, out_path)
#         assert False

## LossPred

In [ ]:
import argparse
import json
import os
import time
import numpy as np
import pickle

def get_slow_coco_outs(
    data_dict, # wavs: boundary dict
    data_json,
    tolerance = 0.05,
    level="syllable",
    block_shift=False,
):
    def get_word_ali(raw_ali):
        """
        raw_ali is a string like 'start1__word1__end1 start2__word2__end2 ...'
        """
        data = []
        meta_toks = raw_ali.split()
        for meta_tok in meta_toks:
            toks = meta_tok.split('__')
            if len(toks) == 3:
                data.append([float(toks[0]), float(toks[2])])
        return data

    def find_boundary_matches(gt, pred, tolerance):
        """
        gt: list of ground truth boundaries
        pred: list of predicted boundaries
        all in seconds
        """
        gt_pointer = 0
        pred_pointer = 0
        gt_len = len(gt)
        pred_len = len(pred)
        match_pred = 0
        match_gt = 0
        while gt_pointer < gt_len and pred_pointer < pred_len:
            if np.abs(gt[gt_pointer] - pred[pred_pointer]) <= tolerance:
                match_gt += 1
                match_pred += 1
                gt_pointer += 1
                pred_pointer += 1
            elif gt[gt_pointer] > pred[pred_pointer]:
                pred_pointer += 1
            else:
                gt_pointer += 1
        return match_gt, match_pred, gt_len, pred_len



    # first run to select systematic shift, using 1/10 of the data
    # following O. Räsänen, G. Doyle, and M. C. Frank, “Pre-linguistic segmentation of speech into syllable-like units,” Cognition, 2018
    tolerance = tolerance # tolerance setting follows the paper
    shifts = np.arange(-0.06, 0.031, 0.005)
    if block_shift:
        shifts = np.arange(-0.0001, 0.0002, 0.0001)
        print(shifts)
    shift_res = {}
    # save the segmentation result when calculating metrics, just in case we want to reuse
    for shift in shifts:
        match_gt_count = 0
        match_pred_count = 0
        gt_b_len = 0
        pred_b_len = 0
        for j, item in enumerate(data_json[:50]):  ## TODO ::10
            wav_key = item['caption']['wav'] if 'caption' in item else item['wav']
            raw_ali = get_word_ali(item[f'{level}_alignment'])
            if len(raw_ali) == 0:
                continue
            gt_b = np.unique(raw_ali).tolist()
            if wav_key not in data_dict:
                print(f"missing {wav_key} in data_dict!")
            pred_b = np.unique(data_dict[wav_key]['word_boundaries']).tolist()
            pred_b = [bb + shift for bb in pred_b]
            a, b, c, d = find_boundary_matches(gt_b, pred_b, tolerance)

            match_gt_count += a
            match_pred_count += b
            gt_b_len += c
            pred_b_len += d
            # if j > 10:
            #     break
        b_prec = match_pred_count / pred_b_len + 1e-5
        b_recall = match_gt_count / gt_b_len + 1e-5
        b_f1 = 2*b_prec*b_recall / (b_prec+b_recall)
        b_os = b_recall / b_prec - 1.
        b_r1 = np.sqrt((1-b_recall)**2 + b_os**2)
        b_r2 = (-b_os + b_recall - 1) / np.sqrt(2)
        b_r_val = 1. - (np.abs(b_r1) + np.abs(b_r2))/2.
        shift_res[shift] = (b_prec, b_recall, b_os, b_f1, b_r_val)

    # select based on max R-val
    all_f1 = [val[4] for val in shift_res.values()]
    ind = np.argmax(all_f1)
    best_shift = list(shift_res.keys())[ind]

    match_gt_count = 0
    match_pred_count = 0
    gt_b_len = 0
    pred_b_len = 0
    for j, item in enumerate(data_json):
        wav_key = item['caption']['wav'] if 'caption' in item else item['wav']
        raw_ali = get_word_ali(item[f'{level}_alignment'])
        if len(raw_ali) == 0:
            continue
        gt_b = np.unique(raw_ali).tolist()
        if wav_key not in data_dict:
            print(f"missing {wav_key} in data_dict!")
        pred_b = np.unique(data_dict[wav_key]['word_boundaries']).tolist()
        pred_b = [bb + best_shift for bb in pred_b]
        a, b, c, d = find_boundary_matches(gt_b, pred_b, tolerance)

        match_gt_count += a
        match_pred_count += b
        gt_b_len += c
        pred_b_len += d
    b_prec = match_pred_count / pred_b_len + 1e-5
    b_recall = match_gt_count / gt_b_len + 1e-5
    b_f1 = 2*b_prec*b_recall / (b_prec+b_recall)
    b_os = b_recall / b_prec - 1.
    b_r1 = np.sqrt((1-b_recall)**2 + b_os**2)
    b_r2 = (-b_os + b_recall - 1) / np.sqrt(2)
    b_r_val = 1. - (np.abs(b_r1) + np.abs(b_r2))/2.

    print("best_shift:", best_shift)
    print("boundary precision:", b_prec)
    print("boundary recall: ", b_recall)
    print("boundary F1:", b_f1)
    print("boundary over-segmentation: ", b_os)
    print("boundary R value: ", b_r_val)
    
    return b_prec, b_recall, b_f1

In [ ]:
from rVADfast import rVADfast
vad = rVADfast()

def get_vad_segment_boundaries(audio_np, sr=16000):
    vad_labels, vad_timestamps = vad(audio_np, sr)
    start, end = vad_labels.nonzero()[0][[0,-1]]
    end = min(end+1, vad_labels.shape[-1])
    start_t, end_t = vad_timestamps[[start, end]]
    return start_t, end_t

def get_vad_full_boundaries(audio_np, sr=16000):
    vad_labels, vad_timestamps = vad(audio_np, sr)
    start, end = vad_labels.nonzero()[0][[0,-1]]
    end = min(end+1, vad_labels.shape[-1])
    start_t, end_t = vad_timestamps[[start, end]]
    return start_t, end_t, (1-vad_labels).nonzero()[0]

In [ ]:
%load_ext cython

In [ ]:

def min_cut(ssm, K, bias=0., min_hop=1, max_hop=25):
    N = ssm.shape[0]

    # cdef double[:,::1] C = np.ones((N, K), dtype=np.float32) * DBL_MAX
    C = np.ones((N+1, K+1, max_hop), dtype=np.double, order="C") * 100000
    B = np.ones((N+1, K+1, max_hop), dtype=np.int32)
    
    for i in range(min_hop, min(max_hop,N)):
        C[i,0,i] = 0.
            
    dp = np.zeros((N+1,N+1), dtype=np.double, order="C")
    cumsum = ssm.cumsum(axis=0).cumsum(axis=1)
    for i in range(N):
        for j in range(N):
            dp[i+1,j+1] = cumsum[i,j] 
    
    for i in range(2*min_hop, N+1):  # exclusive index of last boundary 1 means only hits 0, etc. Start at 2nd cut.
        for l in range(min_hop, max_hop):
            for k in range(1,K):
                best_val = 100000
                best_ind = -1
                for j in range(min_hop, max_hop):
                    a,b,c = i-l-j, i-l, i
                    if a < 0 or b < min_hop:
                        continue
                    on_1 = dp[c,c] - dp[c,b] - dp[b,c] + dp[b,b] 
                    off_1 = dp[c,b] - dp[b,b] - dp[c,a] + dp[b,a] 
                    off_2 = dp[b,c] - dp[a,c] - dp[b,b] + dp[a,b] 
                    on_2 = dp[b,b] - dp[b,a] - dp[a,b] + dp[a,a]
                    
                    on_1 /= (c-b)#*(c-b)
                    off_1 /= np.sqrt((c-b)*(b-a))
                    off_2 /= np.sqrt((b-a)*(c-b))
                    on_2 /= (b-a)#*(b-a)
#                     print(on_1, on_2, off_1, off_2, a,b,c)
#                     print((2/(1/on_1+1/on_2)))
                    
                    cur_val = (C[i-l, k-1, j] + (off_1+off_2)-(on_1+on_2))                    
                    if cur_val < best_val:  # todo add length bias here
                        best_val = cur_val
                        best_ind = l
                    


                B[i,k,l] = best_ind
                C[i,k,l] = best_val
    
    # backtrack
    boundary = []
    prev_b = N
    prev_j = np.argmin(C[N, K-1])
    print(np.min(C[N, K-1]))
    loop = list(range(K))[::-1][:-1]
    boundary.append(prev_b)
#     print(prev_j, np.min(C[N, K-1]))
    for k in loop:
        print(prev_b, k, prev_j, C[prev_b, k, prev_j])
        temp = B[prev_b,k,prev_j]
        prev_b = prev_b - prev_j
        prev_j = temp
        boundary.append(prev_b)
    boundary = boundary[::-1] # reverse
    # boundary = boundary[1:-1] # chop start and end
    # boundary = [item - 0.5 for item in boundary] # adjust
    # boundary[0], boundary[-1] = boundary[0] + 0.5, boundary[-1] + 0.5
    
    return boundary

In [ ]:
@torch.no_grad()
def get_hubert_losses(audio, padding_mask, labels_tensor, mask):
    results = hubert_large_model(
        audio, 
        padding_mask=padding_mask, 
        target_list = [labels_tensor], 
        features_only=False,
        mask_override=mask.cuda(),
    )
    losses = F.cross_entropy(hubert_large_model.get_logits(results, True)[0], labels_tensor[mask]+5, reduction='none')
    return losses

@torch.no_grad()
def get_full_losses(audio, padding_mask, labels_tensor, mask):
    masked_losses = get_hubert_losses(audio, padding_mask, labels_tensor, mask).unsqueeze(0).cpu()
    losses = torch.zeros(mask.shape)
    losses[mask] = masked_losses
    return losses, masked_losses

def hubert_plot_1(key, tensor):
    plot_featmap(None, word_ali=get_word_ali(item[key], fps=50)
         , start_in_sec=0, fps=50, image_already=tensor.cpu().repeat(15,1).numpy())

In [ ]:
window_size = 50  # odd number
ssl = window_size//2
ssr = window_size//2 + 1
spf = 1/50
default_window = 5



def slow_get_hubert_losses(
    audio_file,
    root='/data/scratch/pyp/datasets/coco_pyp/SpokenCOCO/',
    use_vad=True,
    item=None,
):
    audio_file_path = root + audio_file
    
    if use_vad:
        audio, sr = torchaudio.load(audio_file_path)
        start_sec, end_sec, nonzero = get_vad_full_boundaries(audio.view(-1).numpy(), sr)
        audio = audio.cuda()
#         times = item['text_alignment'].split('__')
        min_t, max_t = [int(a*sr) for a in [start_sec, end_sec]]
        audio = audio[:,min_t:max_t]
        
        LAYER=9
        base_results = get_hubert_results(audio)
        base_layer_result = base_results['hidden_states'][LAYER][0].transpose(0,1)[0]
        labels_tensor = hubert_akm(base_layer_result).cuda()
        T = labels_tensor.size(-1)
        padding_mask = audio.new_zeros(audio.shape[:2], dtype=bool)
    else:
        # Get labels
        LAYER=9  # todo faster
        base_results = get_hubert_results(audio_file_path)
        base_layer_result = base_results['hidden_states'][LAYER][0].transpose(0,1)[0]
        
        labels_tensor = hubert_akm(base_layer_result).cuda()
        T = labels_tensor.size(-1)

        # Get hubert loss
        audio, sr = torchaudio.load(audio_file_path)
        audio = audio.cuda()
        padding_mask = audio.new_zeros(audio.shape[:2], dtype=bool)

    mask = torch.zeros(1, T).bool()
    full_losses = torch.zeros(T,T)  # from, to
    default_loss = torch.zeros(T)
    default_count = torch.zeros(T)
    
    for mid in tqdm(range(-ssl,T+ssr)):
        mask = torch.zeros((T,)).bool()
        mask[max(0,mid-ssl):min(T,mid+ssr)] = 1
        losses, masked_losses = get_full_losses(audio, padding_mask, labels_tensor, mask.unsqueeze(0))

        if mid-ssl-1 >= 0:
            losses1 = losses.clone().view(-1)
            losses1[min(T,mid):] = 0
            full_losses[mid-ssl-1] += losses1

        if mid+ssr < T:
            losses2 = losses.clone().view(-1)
            losses2[:max(0,mid)] = 0
            full_losses[mid+ssr] += losses2

        cur_default_loss = losses.clone()[0]
        cur_default_loss[:max(0,mid-default_window)] = 0
        cur_default_loss[min(T, max(0,mid+default_window+1)):] = 0
        default_loss += cur_default_loss
        default_count += (cur_default_loss > 0).int()
        
    default_losses = (default_loss / default_count).unsqueeze(0)
    
    return full_losses, default_losses, base_layer_result, audio, start_sec, end_sec, nonzero, base_layer_result

In [ ]:
hubert_large_model.half()
@torch.inference_mode()
def fast_get_hubert_losses(
    audio_file,
    root='/data/scratch/pyp/datasets/coco_pyp/SpokenCOCO/',
    item=None,
    window_size = 51,  # odd number
    batch_size = 32,
    gap_size = None,
):
    ssl = window_size//2
    ssr = window_size//2+1
    spf = 1/50
    default_window = 5
    LAYER=9
    
    audio_file_path = root + audio_file
    
    audio, sr = torchaudio.load(audio_file_path)
    start_sec, end_sec, nonzero = get_vad_full_boundaries(audio.view(-1).numpy(), sr)
    audio = audio.cuda()
    min_t, max_t = [int(a*sr) for a in [start_sec, end_sec]]
    audio = audio[:,min_t:max_t]

    base_results = get_hubert_results(audio)
    base_layer_result = base_results['hidden_states'][LAYER][0].transpose(0,1)[0]
    labels_tensor = hubert_akm(base_layer_result).cuda()
    T = labels_tensor.size(-1)
    padding_mask = audio.new_zeros(audio.shape[:2], dtype=bool)

    full_losses = torch.zeros(T,T)  # from, to
    default_loss = torch.zeros(T)
    default_count = torch.zeros(T)
    N = T+ssr+ssl-1
    all_losses = torch.zeros(N,T)  # from, to
        
    if gap_size is not None and gap_size >= T:
        gap_size = None
    
    if gap_size is None:
        mask = torch.zeros(N,T,dtype=bool)
        for idx, mid in enumerate(range(-ssl,T+ssr-1)):
            mask[idx, max(0,mid-ssl):min(T,mid+ssr)] = True

        audio = audio.half()

        for idx in range(0, N, batch_size):
            bs = min(N, idx+batch_size) - idx
            losses, _ = get_full_losses(audio.expand(bs,-1), padding_mask.expand(bs,-1), labels_tensor.expand(bs,-1), mask[idx:idx+bs])
            all_losses[idx:idx+bs] = losses
    else:
        mask = torch.zeros(gap_size,T,dtype=bool)
        
        for idx, mid in enumerate(range(-ssl,-ssl+gap_size)):
            for mid2 in range(mid, T+ssr-1, gap_size):
                mask[idx, max(0,mid2-ssl):min(T,mid2+ssr)] = True
        audio = audio.half()

        for idx in range(0, gap_size, batch_size):
            bs = min(gap_size, idx+batch_size) - idx
            
            cur_mask = mask[idx:idx+bs].cuda()
            cur_labels = labels_tensor.expand(bs,-1)
            features_only = hubert_large_model(
                audio.expand(bs,-1), 
                padding_mask=padding_mask.expand(bs,-1), 
                target_list = [cur_labels], 
                features_only=True,
                mask_override=cur_mask,
            )
            
            
            features = hubert_large_model.final_proj(features_only['x'])
            x = (features/((features**2).sum(dim=-1, keepdim=True).sqrt()+1e-5) @ 
                 (hubert_large_model.label_embs_concat/((hubert_large_model.label_embs_concat**2)
                                                        .sum(dim=-1, keepdim=True).sqrt() + 1e-5)).unsqueeze(0).transpose(-1,-2)
                )
            x = x / hubert_large_model.logit_temp
            losses = x.new_zeros(cur_mask.shape)
            losses[cur_mask] = F.cross_entropy(x[cur_mask], cur_labels[cur_mask]+4, reduction='none')
            
            for idx2 in range(idx, N, gap_size):
                all_losses[idx2:min(N,idx2+bs)] = losses[:min(bs, N-idx2)]
        
    
    for idx, mid in enumerate(range(-ssl,T+ssr-1)):
        losses = all_losses[idx]
        if mid-ssl-1 >= 0:
            losses1 = losses.clone().view(-1)
            losses1[min(T,mid):] = 0
            full_losses[mid-ssl-1] += losses1

        if mid+ssr < T:
            losses2 = losses.clone().view(-1)
            losses2[:max(0,mid)] = 0
            full_losses[mid+ssr] += losses2

        cur_default_loss = losses.clone()
        cur_default_loss[:max(0,mid-default_window)] = 0
        cur_default_loss[min(T, max(0,mid+default_window+1)):] = 0
        default_loss += cur_default_loss
        default_count += (cur_default_loss > 0).int()
        
    full_losses = full_losses.tril(ssr+2).triu(-(ssr+2))
    default_losses = (default_loss / default_count).unsqueeze(0)
    
    return full_losses, default_losses, base_layer_result, audio, start_sec, end_sec, nonzero

In [ ]:
# Getting a 2d loss matrix efficiently from off-the-shelf HuBERT models

t1 = time.time()
full_losses, default_losses, base_layer_result, audio, start_sec, end_sec, nonzero = fast_get_hubert_losses(
    item['caption']['wav'], 
    root="", 
    batch_size=32,
    gap_size=150,
)
print(time.time()-t1)

In [ ]:
%%cython

cimport cython
import numpy as np
cimport numpy as np
np.import_array()
from libc.math cimport sqrt
from cython.operator import postincrement as inc
from cpython.mem cimport PyMem_Malloc, PyMem_Free

cdef extern from "float.h":
    double DBL_MAX

    
    
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def min_cut_c(np.ndarray ssm, Py_ssize_t Ks, double bias=0., min_hop=1, allow_skip=0, weird=0.075, can_skip=0):
    cdef Py_ssize_t K = np.max(Ks)
    cdef Py_ssize_t N = ssm.shape[0]

    # cdef double[:,::1] C = np.ones((N, K), dtype=np.float32) * DBL_MAX
    cdef double[:,::1] C = np.ones((N+1, K), dtype=np.float32, order="C") * DBL_MAX
    cdef int[:,::1] B = np.ones((N+1, K), dtype=np.int32)

    C[0,0] = 0.
#     C[0,0] = -np.inf
    
    cdef list temp, obj
    cdef Py_ssize_t i, j, k, ind, a
    
    # kinda slow copy
    cdef double[:,:] dp = np.zeros((N+1,N+1), dtype=np.double, order="C")
    cumsum = ssm.cumsum(axis=0).cumsum(axis=1)
    for i in range(N):
        for j in range(N):
            dp[1+i,1+j] = cumsum[i,j] 

    for i in range(min_hop, N+1-allow_skip):
        temp = [(
            (dp[i,i]-dp[j,i]-dp[i,j]+dp[j,j]),  # interior
            (dp[i,j]-dp[j,j])*1.0,  # left  0.75
            (dp[j,i]-dp[j,j])*1.0,  # top
            (dp[N,i]-dp[i,i]-dp[N,j]+dp[i,j])*1.0,  # bottom
            (dp[i,N]-dp[i,i]-dp[j,N]+dp[j,i])*1.0,  # right
            ) for j in range(i-min_hop+1)
        ]
        for k in range(1,K):
            obj = [C[max(0,j-can_skip), k-1] + (((item[1]+item[2]+item[3]+item[4])/2.)/((item[1]+item[2]+item[3]+item[4])/2.+item[0]/2.+1e-5)) for j, item in enumerate(temp)]
            ind = np.argmin(obj)
            B[i+allow_skip,k] = ind
            C[i+allow_skip,k] = obj[ind]

    
    # backtrack
    cdef list boundary = []
#     cdef Py_ssize_t prev_b = N - 1
    cdef Py_ssize_t prev_b = N
#     while K > 5 and C[N,K-1]/K >= weird:
#         K -= 1
    cdef list loop = list(range(K))[::-1][:-1]
    boundary.append(prev_b)
    for k in loop:
        prev_b = B[prev_b,k]
        boundary.append(prev_b)
    boundary = boundary[::-1] # reverse
    # boundary = boundary[1:-1] # chop start and end
    # boundary = [item - 0.5 for item in boundary] # adjust
    # boundary[0], boundary[-1] = boundary[0] + 0.5, boundary[-1] + 0.5
    return boundary, C[N,K-1]*1.0/K

In [ ]:
%%cython

cimport cython
import numpy as np
cimport numpy as np
np.import_array()
from libc.math cimport sqrt
from cython.operator import postincrement as inc
from cpython.mem cimport PyMem_Malloc, PyMem_Free

cdef extern from "float.h":
    double DBL_MAX

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def min_cut_efficient_c(np.ndarray dp, Py_ssize_t K, min_hop=3, max_hop=40):
    cdef Py_ssize_t N = dp.shape[0] - 1

    cdef double[:,::1] C = np.ones((N+1, K), dtype=np.float32, order="C") * DBL_MAX
    cdef int[:,::1] B = np.ones((N+1, K), dtype=np.int32)
    C[0,0] = 0.
    
    cdef list temp, obj
    cdef Py_ssize_t i, j, k, ind, a, best_ind
    cdef double best_val, cur_val
    
    for i in range(min_hop, N+1):
        temp = []
        for j in range(max(0, i-max_hop+1), i-min_hop+1):
            item = (
                (dp[i,i]-dp[j,i]-dp[i,j]+dp[j,j]),  # interior
                (dp[i,j]-dp[j,j]),  # left
                (dp[j,i]-dp[j,j]),  # top
                (dp[N,i]-dp[i,i]-dp[N,j]+dp[i,j]),  # bottom
                (dp[i,N]-dp[i,i]-dp[j,N]+dp[j,i]),  # right
            )
            temp.append((j, (item[1]+item[2]+item[3]+item[4])/(item[1]+item[2]+item[3]+item[4]+item[0]+1e-5)))
        for k in range(1,K):
            obj = [C[j, k-1] + item for (j, item) in temp]
            ind = np.argmin(obj)
            B[i,k] = temp[ind][0]
            C[i,k] = obj[ind]

    
    cdef list boundary = []
    cdef Py_ssize_t prev_b = N
    cdef list loop = list(range(K))[::-1][:-1]
    boundary.append(prev_b)
    for k in loop:
        prev_b = B[prev_b,k]
        boundary.append(prev_b)
    boundary = boundary[::-1]

    return boundary, C[N,K-1]

In [ ]:
def min_cut_efficient(dp, Ks, min_hop=3, max_hop=50):
    K = max(Ks)
    N = dp.shape[0] - 1

    C = np.ones((N+1, K), dtype=np.float32, order="C") * 100000
    B = np.ones((N+1, K), dtype=np.int32)
    C[0,0] = 0.
    
    for i in range(min_hop, N+1):
        temp = []
        for j in range(max(0, i-max_hop+1), i-min_hop+1):
            item = (
                (dp[i,i]-dp[j,i]-dp[i,j]+dp[j,j]),  # interior
                (dp[i,j]-dp[j,j]),  # left
                (dp[j,i]-dp[j,j]),  # top
                (dp[N,i]-dp[i,i]-dp[N,j]+dp[i,j]),  # bottom
                (dp[i,N]-dp[i,i]-dp[j,N]+dp[j,i]),  # right
            )
            temp.append((j, (item[1]+item[2]+item[3]+item[4])/(item[1]+item[2]+item[3]+item[4]+item[0]+1e-5)))
        for k in range(1,K):
            obj = [C[j, k-1] + item for (j, item) in temp]
            ind = np.argmin(obj)
            B[i,k] = temp[ind][0]
            C[i,k] = obj[ind]

    
    boundaries = []
    for K in Ks:
        boundary = []
        prev_b = N
        loop = list(range(K))[::-1][:-1]
        boundary.append(prev_b)
        for k in loop:
            prev_b = B[prev_b,k]
            boundary.append(prev_b)
        boundary = boundary[::-1]
        
        boundaries.append(boundary)

    return boundaries


In [ ]:
# Efficient loss pred extraction code from precalculated losses

def clean_log_mi_exp(nlog_mi, min_clip=-10):
    log_mi = -nlog_mi
    for row in range(log_mi.size(0)):
        log_mi[row, min(log_mi.size(0), row+ssr):] = min_clip
        log_mi[row, :max(0, row-ssl)] = min_clip
        log_mi[row, row] = min_clip    
    log_mi = log_mi.clamp(min=min_clip).exp()
    
    return log_mi


def set_diagonal(x, v, dist=None):
    x = x.clone()
    for row in range(x.size(0)):
        x[row, min(x.size(0), row+(ssr if dist is None else dist+1)):] = v
        x[row, :max(0, row-(ssl if dist is None else dist))] = v
    return x

def process_loss_multiple(
    full_losses, 
    default_losses,
    b1 = 0,
    b2 = 0,
    m1 = 1,
    m11 = 1,
    m2 = 0,
    m21 = 0,
    m3 = 1,
    m4 = 1,
    y1 = 0.5,
    y2 = 0.5,
    y3 = 0.5,
    y4 = 0.0,
    y5 = 0.0,
    sec_per_syllables = [0.15],
    sec_shift = 0,
    max_clip = =100,
    mincut_bias = 0,
    mincut_minhop = 1,
    silent_sec = None,
):
    full_losses = set_diagonal(full_losses, -max_clip/m2)
    gain = full_losses

    T = gain.size(0)
    ks = [int(T / 50 / sec_per_syllable) + 1 for sec_per_syllable in sec_per_syllables]
    spf = 1/50

    cur_feats = clean_log_mi_exp(gain, min_clip=max_clip)

    dp = np.zeros((cur_feats.size(0) + 1, cur_feats.size(1)+1))
    dp[1:,1:] = cur_feats.cumsum(dim=0).cumsum(dim=1)

    min_cut_boundaries = min_cut_efficient(dp, ks, mincut_minhop)
    
    word_boundarieses = []
    for min_cut_boundary in min_cut_boundaries:    
    
        word_boundaries = [[l*spf+sec_shift,r*spf+sec_shift] for l, r in zip(min_cut_boundary[:-1], min_cut_boundary[1:])]

        if silent_sec is not None:
            word_boundaries = [wb for wb in word_boundaries if abs(wb[0]-silent_sec[np.searchsorted(silent_sec, wb[0])]) > 0.001]

        word_boundarieses.append(word_boundaries)
            
    return word_boundarieses

In [ ]:
### Evaluate LossPred Over a Dataset

data_dict = {}

CUR_DATA= ls_aligned_dev[:500]

abc = []
for item in tqdm(CUR_DATA):
    a,b,_,audio,c,d,e = fast_get_hubert_losses(item['caption']['wav'], root="", gap_size=150)
    word_boundaries = process_loss_multiple(
        a,
        b,
        sec_shift=c,
        m1=0.5,
        m2=0.5,
        y1=0.0,
        y2=0.0,
        sec_per_syllables=[0.16], # safe
        mincut_minhop=1,
        max_clip=-100,
        silent_sec = e/100.
    )[0]
    
    data_dict[item['caption']['wav']] = {'word_boundaries': word_boundaries}
    _ = word_boundaries[-1][-1]/len(word_boundaries)
    abc.append(_)
    
get_slow_coco_outs(data_dict, CUR_DATA, level='syllable', tolerance=0.05)